In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
import keras
from keras import losses, optimizers, metrics

/Users/alex/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class ProteinLigandDataGenerator(keras.utils.Sequence):
    def __init__(self, csv_path, batch_size, shuffle=True, dim=(24,24,24), n_channels=16):
        self.df = self.prepare_df(csv_path)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = None
        self.dim = dim
        self.n_channels = n_channels
        self.on_epoch_end()
    
    def prepare_df(self, csv_path):
        # oversampling happens here, 
        correct_count = 1
        random_count = 4
        supplement = correct_count - random_count
        
        df = pd.read_csv(csv_path)
        same_idx = df[df['pro_id'] == df['lig_id']]
        return df.append([same_idx]*supplement, ignore_index=True)
        
    
    def __len__(self):
        '''
        Batches per epoch
        '''
        return int(len(self.df.index)/self.batch_size)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        X, y = self.generate_data(indexes)
        
        return X, y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df.index))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def generate_data(self, indexes):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        sub_df = self.df.iloc[indexes]
        sub_df.reset_index(inplace=True)
        for row in sub_df.itertuples():
            idx = row[0]
            X[idx,] = np.load(row.dests)
            y[idx] = row.score
            
        return X, y

In [3]:
train_generator = ProteinLigandDataGenerator('./data/csv/train_lig_pro_pairs.csv', 12)
test_generator = ProteinLigandDataGenerator('./data/csv/test_lig_pro_pairs.csv', 12)

In [4]:
model = Sequential()
model.add(Conv3D(96, kernel_size=3, activation='relu', input_shape=(24,24,24,16)))
model.add(BatchNormalization())
model.add(Conv3D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool3D())
model.add(Conv3D(196, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv3D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling3D())
model.add(Dense(1, activation='sigmoid'))

sgd = optimizers.SGD(lr=1, decay=1e-5)
model.compile(loss=losses.mean_squared_error, optimizer=sgd, metrics=[metrics.mse])

In [5]:
history = model.fit_generator(train_generator, epochs=2, use_multiprocessing=True, workers=4, verbose=1)

Epoch 1/2
1800/1800 [==============================] - 549s 305ms/step - loss: 0.0329 - mean_squared_error: 0.0329
Epoch 2/2
1800/1800 [==============================] - 587s 326ms/step - loss: 0.0194 - mean_squared_error: 0.0194


In [6]:
test_history = model.evaluate_generator(test_generator, use_multiprocessing=True, workers=4)

In [15]:
print('train: ', history.history['mean_squared_error'])
print('test: ', test_history)

train:  [0.032865592476587144, 0.019414262728500564]
test:  [0.015346607277060685, 0.015346607277060685]


In [11]:
model.save('models/basic_regression_model_beta.h5')